In [1]:
# Install two libraries used to create a GUI and host it online on colab
!pip install streamlit
!pip install pyngrok

     |████████████████████████████████| 9.9 MB 4.1 MB/s 
     |████████████████████████████████| 181 kB 19.5 MB/s 
     |████████████████████████████████| 4.3 MB 42.3 MB/s 
     |████████████████████████████████| 111 kB 44.2 MB/s 
     |████████████████████████████████| 164 kB 47.1 MB/s 
     |████████████████████████████████| 76 kB 5.2 MB/s 
     |████████████████████████████████| 63 kB 1.8 MB/s 
     |████████████████████████████████| 128 kB 50.8 MB/s 
     |████████████████████████████████| 793 kB 52.7 MB/s 
     |████████████████████████████████| 380 kB 58.9 MB/s 
  Created wheel for blinker: filename=blinker-1.4-py3-none-any.whl size=13478 sha256=3887f9027e70fb2ca70ad1dfe05bd3223851c94b2702b95217fcdd918bb31759
  Stored in directory: /root/.cache/pip/wheels/22/f5/18/df711b66eb25b21325c132757d4314db9ac5e8dabeaf196eab
Successfully built blinker
  Attempting uninstall: prompt-toolkit
    Found existing installation: prompt-toolkit 1.0.18
    Uninstalling prompt-toolkit-1.0.18:
      S

     |████████████████████████████████| 745 kB 3.2 MB/s 
  Created wheel for pyngrok: filename=pyngrok-5.1.0-py3-none-any.whl size=19007 sha256=275d3a041454c42207f2da242b5a0abe1d130cdab63f0b288319604f7a199829
  Stored in directory: /root/.cache/pip/wheels/bf/e6/af/ccf6598ecefecd44104069371795cb9b3afbcd16987f6ccfb3
Successfully built pyngrok


In [9]:
## Unzipping the following code to unzip any imported models.

!unzip /content/64X3-CNN.model.zip
# !unzip /content/CNN-RF.model.zip

Archive:  /content/64X3-CNN.model.zip
   creating: 64X3-CNN.model/
replace __MACOSX/._64X3-CNN.model? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: __MACOSX/._64X3-CNN.model  
  inflating: 64X3-CNN.model/keras_metadata.pb  
replace __MACOSX/64X3-CNN.model/._keras_metadata.pb? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: __MACOSX/64X3-CNN.model/._keras_metadata.pb  
   creating: 64X3-CNN.model/variables/
replace __MACOSX/64X3-CNN.model/._variables? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: __MACOSX/64X3-CNN.model/._variables  
  inflating: 64X3-CNN.model/saved_model.pb  
replace __MACOSX/64X3-CNN.model/._saved_model.pb? [y]es, [n]o, [A]ll, [N]one, [r]ename: a
error:  invalid response [a]
replace __MACOSX/64X3-CNN.model/._saved_model.pb? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: __MACOSX/64X3-CNN.model/._saved_model.pb  
   creating: 64X3-CNN.model/assets/
  inflating: __MACOSX/64X3-CNN.model/._assets  
  inflating: 64X3-CNN.model/variables/variables.

In [19]:
## App coding starts from here

%%writefile app.py
import streamlit as st
from PIL import Image
import cv2
import numpy as np
import tensorflow as tf
import pandas as pd

# Use the Image Data Generator to import the images from the dataset
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.inception_v3 import preprocess_input
from tensorflow.keras.preprocessing import image

PAGE_CONFIG = {"page_title":"Group 32 - KU","page_icon":":smiley:","layout":"centered"}
st.set_page_config(**PAGE_CONFIG)

Image_size = 100
Results = ['Normal',"Anomaly"]

team_members = ['Deepak Jangir Dayanand','Sagrika Bhandari','Shweta Nair','Preshit Dwivedi']
team_members_id = ['K2161089','K2145485','K2144813','K2145296']
df = pd.DataFrame({"Team Members": team_members,
                   "Team ID's": team_members_id})

df = df.set_index("Team ID's")

## Importing all the models implemented
model1 = tf.keras.models.load_model('64X3-CNN.model')
model3 = tf.keras.models.load_model('CNN_SVM.h5')
modelresnet = tf.keras.models.load_model('model_resnet50.h5')
modelrf = tf.keras.models.load_model('CNN-RF.model')

def load_image(image_file):
  img = Image.open(image_file)
  return img

#Required for Model 1 - CNN
def convert_to_image_array_opencv(image_file):

  img = Image.open(image_file)
  img = img.save("img.jpg")

  # OpenCv Read
  opencv_image = cv2.imread("img.jpg", cv2.IMREAD_GRAYSCALE)
  Resized_array = cv2.resize(opencv_image,(Image_size, Image_size))

  return Resized_array.reshape(-1,Image_size,Image_size,1)

#Required for Model 3 - SVM
def predict_anomaly_SVM(image_file):
  img = Image.open(image_file)
  img = img.save("img-SVM.jpg")

  test_image = image.load_img("img-SVM.jpg", target_size = (64,64))
  test_image = image.img_to_array(test_image)
  test_image = test_image/255
  test_image = np.expand_dims(test_image, axis = 0)
  result = model3.predict(test_image)
  
  return result


def display_image_details_and_image(uploaded_image_file):

  file_details = {"File Name":uploaded_image_file.name, "File Type":uploaded_image_file.type,
                        "File Size":uploaded_image_file.size}
  st.write(file_details)
  st.image(load_image(uploaded_image_file),width=250)


def main():
    st.title("Image Anamoly Detection Models using Deep Learning")
 
    with st.sidebar:
      st.subheader("Deep Learning Trained Models")
      model_names = ["Explore Models",
                     "Model 1 - Designed CNN",
                     "Model 2 - DL using SVM",
                     "Model 3 - Using CNN filter and RF",
                     "Model 4 - Transfer Learning - resnet50"]
      choice = st.sidebar.selectbox(label = "Choose a Model",options=model_names)

      st.sidebar.table(df)

    if choice == 'Explore Models':
        st.subheader("Choose a Model to start detecting Anomalies")

    elif choice == 'Model 1 - Designed CNN':
      st.subheader("A Convolutional neural network model built using 2 Dense layers, 3 Convolutional layers having the layer sizes as 64")
      uploaded_image_file = st.file_uploader("Upload your Image to be tested", type=["png","jpg","jpeg"])

      if uploaded_image_file is not None:

        display_image_details_and_image(uploaded_image_file)

        model_prediction = model1.predict([convert_to_image_array_opencv(uploaded_image_file)])

        if int(model_prediction) == 0:
          st.success("The Image is found to be : " + Results[int(model_prediction)])
        else:
          st.error("The Image is found to be : " + Results[int(model_prediction)])

    elif choice == 'Model 2 - DL using SVM':
      st.subheader("In this model, the SVM classifier is used in a convolutional neural network to do the Anomaly Detection")
      uploaded_image_file = st.file_uploader("Upload your Image to be tested", type=["png","jpg","jpeg"])

      if uploaded_image_file is not None:

        display_image_details_and_image(uploaded_image_file)

        model_prediction = predict_anomaly_SVM(uploaded_image_file)

        if model_prediction[0] < 0:
          st.error("The Image is found to be : Anomaly")
        else:
          st.success("The Image is found to be : Normal")

    elif choice == 'Model 3 - Using CNN filter and RF':
      st.subheader("The main aim in this model was to make use of the Convolutional Neural Networks in such a way, that its filters could be extracted and later use them as input features to a Traditional Random Forest Classifier in order to detect anomalies in an Image.")
      uploaded_image_file = st.file_uploader("Upload your Image to be tested", type=["png","jpg","jpeg"])

      if uploaded_image_file is not None:

        display_image_details_and_image(uploaded_image_file)

        model_prediction = model1.predict([convert_to_image_array_opencv(uploaded_image_file)])

        if int(model_prediction) == 0:
          st.success("The Image is found to be : " + Results[int(model_prediction)])
        else:
          st.error("The Image is found to be : " + Results[int(model_prediction)])

    elif choice == 'Model 4 - Transfer Learning - resnet50':
      st.subheader("For the analysis of the existing DNN’s we used Transfer Learning by making use of the Resnet50 architecture. We utilised the weights which were obtained when the model was trained on the Imagenet data set.")
      uploaded_image_file = st.file_uploader("Upload your Image to be tested", type=["png","jpg","jpeg"])

      if uploaded_image_file is not None:

        display_image_details_and_image(uploaded_image_file)

        img = Image.open(uploaded_image_file)
        img = img.save("img-RF.jpg")


        img=image.load_img("img-RF.jpg",target_size=(224,224))
        x=image.img_to_array(img)
        x=x/255

        x=np.expand_dims(x,axis=0)
        img_data=preprocess_input(x)

        a=np.argmax(modelresnet.predict(img_data), axis=1)

        st.success("The Image is found to be : " + Results[int(a)])
        # st.write(Results[int(a)])

  
if __name__ == '__main__':
    main()

Overwriting app.py


In [8]:
## Use this block to run the pyngrok and open the streamlit app on a server whose http link will be displayed below in the output

from pyngrok import ngrok

!ngrok authtoken 261WeU8cQS95u1eoGredriNItwn_2ZsF6FKumDXm8H1geNLeo

!streamlit run --server.port 80 app.py &>/dev/null&

publ_url = ngrok.connect(port='8501')

publ_url 

Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


<NgrokTunnel: "http://6d4f-35-231-211-58.ngrok.io" -> "http://localhost:80">

In [2]:
!pgrep streamlit

In [3]:
ngrok.kill()

NameError: ignored